# EKE

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

## Set parameters

In [ ]:
# Time
start = "1993-01"
stop = "2022-12"

# Region
lat_slice = slice(-65, 65)
lon_slice = slice(-180, 180)

# Versions
versions = ["vdt2021", "vdt2024"]

## Define request

In [ ]:
collection_id = "satellite-sea-level-global"
request = {"variable": "daily"}

## Define functions to cache

In [ ]:
def compute_eke(ds, lon_slice, lat_slice):
    ds = utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice)
    eke = 1 / 2 * 1.0e4 * (ds["ugosa"] ** 2 + ds["vgosa"] ** 2)
    eke = eke.resample(time="1MS").mean()
    eke.attrs = {
        "long_name": "Eddy Kinetic Energy",
        "units": "cm^2 s^{-2}",
        "description": "Eddy Kinetic Energy from geostrophic velocity anomalies (ugosa, vgosa)",
    }
    return eke.to_dataset(name="EKE")

## Download and transform

In [ ]:
datasets = []
for version in versions:
    print(f"{version=}")
    requests = download.update_request_date(
        request | {"version": version}, start, stop, stringify_dates=True
    )
    ds = download.download_and_transform(
        collection_id,
        requests,
        chunks={"year": 1, "month": 1},
        transform_func=compute_eke,
        transform_func_kwargs={"lon_slice": lon_slice, "lat_slice": lat_slice},
    )
    datasets.append(ds.expand_dims(version=[version]))
ds = xr.concat(datasets, "version").mean("time", keep_attrs=True)

## Quick and dirty plot

In [ ]:
plot.projected_map(ds["EKE"], col="version", robust=True)